、# Newton Method

In [1]:
newton(f, f′, x) = x - f(x) / f′(x)

newton (generic function with 1 method)

In [2]:
f(x) = x^2 - 2
f′(x) = 2x

x0 = 1.0
x = x0

for i in 1:10
    x = newton(f, f′, x)
    println(x)
end

1.5
1.4166666666666667
1.4142156862745099
1.4142135623746899
1.4142135623730951
1.414213562373095
1.4142135623730951
1.414213562373095
1.4142135623730951
1.414213562373095


In [3]:
f(x) = x^2 - 2
f′(x) = 2x

f′ (generic function with 1 method)

In [5]:
sqrt(2)

1.4142135623730951

In [9]:
function do_newton(g, g′, x0, max_iterations=100)  # valeur par defaut
    
    x = x0

    tolerance = 1e-15
    iteration = 0

    while ( abs(g(x) - 0) > tolerance 
        && iteration < max_iterations )
        
        x = newton(g, g′, x)
        
        iteration += 1
        
#         if iteration == max_iterations
#             break
#         end
    end
    
    return x , g(x)
end

do_newton (generic function with 2 methods)

In [10]:
x, y = do_newton(f, f′, 100);

In [11]:
x

1.4142135623730951

In [12]:
y

4.440892098500626e-16

In [13]:
h(z) = z^3 - 1
h′(z) = 3z^2

do_newton(h, h′, 0.5+7im)

(-0.5 + 0.8660254037844386im,-2.220446049250313e-16 + 1.1102230246251565e-16im)

# Airline Network Revenue Management

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

In the airline network revenue management problem we are trying to decide how many tickets for each origin-destination (O-D) pair to sell at each price level. The goal is to maximize revenue, and we cannot sell more tickets than there is demand for, or space on the planes for.

## Three Flight Problem

We'll start with a toy problem that has three origin-destination pairs, with two price classes for each pair. The three origin-destination pairs are BOS-MDW, MDW-SFO, or BOS-SFO via MDW. BOS stands for Boston, MDW is Chicago-Midway, and SFO is San Francisco. Each O-D pair has a "regular" and "discount" fare class. The data we will use is summarized as follows:

```
PLANE CAPACITY: 166

BOS-MDW
        Regular  Discount
Price:  428      190
Demand: 80       120

BOS-SFO
        Regular  Discount
Price:  642      224
Demand: 75       100

MDW-SFO
        Regular  Discount
Price:  512      190
Demand: 60       110
```

In [17]:
# First we need to load the JuMP package
using JuMP

### Model
Now we need to create a `Model`. A `Model` is an object that has associated variables and constraints. We can also pick and customize different solvers to use with the model. In this case we'll assume you a LP solver installed - JuMP will detect it and use it automatically.

In [18]:
nrm = Model()

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

We can see that JuMP displays the model in a human-readable form.

### Variables
Now we can create our variables, in the optimization sense. Variables have a name, bounds, and type. For this problem, we need six continuous variables, each with a lower and upper bound on their value.

Here we will spell out each variable one-by-one - rest assured that later we will do something smarter that will scale to millions of variables!

In [20]:
@variables(nrm, begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end)
nrm

Feasibility problem with:
 * 0 linear constraints
 * 6 variables
Solver is default solver

Great, now we are getting somewhere!

### Objective
The objective is to maximize the revenue. We set the objective with `@objective`, which takes three arguments: the model, the sense (`Max` or `Min`), and the expression.

In [21]:
@objective(nrm, Max, 428*BOStoMDW_R + 190*BOStoMDW_D +
                        642*BOStoSFO_R + 224*BOStoSFO_D +
                        512*MDWtoSFO_R + 190*MDWtoSFO_D)
nrm

Maximization problem with:
 * 0 linear constraints
 * 6 variables
Solver is default solver

### Constraints
We have only two constraints, one per physical flight: that the number of people on each flight is less than the capacity of the planes. 

We add constraints with `@constraint`, which takes two arguments: the model, and an expression with an inequality in it: `<=`, `==`, `>=`.

(note that there are other forms of `@constraint` that can be useful sometimes - see the manual or other examples for details)

In [22]:
@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                    BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                    BOStoSFO_R + BOStoSFO_D <= 166)
nrm

Maximization problem with:
 * 2 linear constraints
 * 6 variables
Solver is default solver

Our model is complete!

## Solve
The easy part! Lets check out the finished model before solving it. We didn't specify a solver before, but JuMP knows we have an LP solver installed, so it will use that.

In [23]:
status = solve(nrm)
status  # Should be `:Optimal`

:Optimal

## Inspect the solution
We can use `getvalue()` to inspect the value of solutions

In [24]:
getvalue(BOStoMDW_R)

80.0

In [25]:
getvalue(BOStoMDW_D)

11.0

In [26]:
getobjectivevalue(nrm)

121090.0

In [ ]:
getvalue(BOStoMDW_R)